In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# URL of the webpage to scrape
url = "https://dlai-lc-dag.s3.us-east-2.amazonaws.com/apartment_finder.html"

### START CODE HERE ###

# send a GET request to the url
response = requests.get(url)

# get the status of the response for troubleshooting
status = response.status_code

### END CODE HERE ###

print(status)

200


In [4]:
print(response.text)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Welcome to CityDwellers</title>
    <link rel="stylesheet" href="styles_green.css">
</head>
<body>

    <div class="content">
        <header>
            <h1>Welcome to CityDwellers - Your Ultimate Apartment Finder</h1>
        </header>
        <section>
            <p>At CityDwellers, we believe that finding your dream apartment should be exciting, not exhausting. Our platform is designed to simplify your search, offering a seamless experience from browsing to signing the lease. Whether you're looking for a cozy studio, a spacious family home, or a luxurious penthouse, your perfect match is just a few clicks away.</p>
        </section>
        <section>
            <h2>Why Choose CityDwellers?</h2>
            <ul>
                <li>Vast Selection: Browse thousands of listings in prime locations. From bustling city centers to qui

In [5]:
### START CODE HERE ###

# Parse the HTML content
soup = BeautifulSoup(response.text)

# Find all the grid items
grid_items = soup.find_all("div", class_="grid-item")

### END CODE HERE ###

# Print the number of grid items that were found
print(len(grid_items))

72


<a id="extract"></a>

## Step 4: Extract the elements and create a pandas DataFrame

As the next step, you will search through the `grid_items` to extract individual pieces of information about the apartments. Notice that each piece of information is stored in its own paragraph `<p>` and you can find all the paragraphs using `item.find_all("p")`, much the same way as you found all `grid_items` before. This code is provided for you. The `paragraphs` variable then stores a list of all paragraphs and you can access them one by one using indices. For this task, you need to extract the data to create a pandas DataFrame. Four of the columns are already extracted and the rest are left for you to be finished. In order to finish this lab, you need to at least populate the `parking` column, as you will need this one for further analysis. You can populate the rest of the columns for practice.

Since each line of code has quite a lot to unpack, here is a breakdown of what happens in the `number_of_bedrooms` column. This is the code used to extract the value:

`"number_of_bedrooms": paragraphs[0].text.split(" ")[0].strip()`

As you can see this is a dictionary entry, where `number_of_bedrooms` is the key and the rest is the value of this specific entry. First you access the first item in the `paragraphs` list by using `paragraphs[0]`. then you use `.text` to access the text within this paragraph. This gives you only the text without any HTML tags, returning for example `2 Bedrooms`. As you just want to keep the number, you can use `.split(" ")` to split it by the whitespace and then select the first element in the split by using `[0]`. In the end you use `.strip()` to remove any whitespace that may be left.

In this image you can see which index in the `paragraphs` variable corresponds to each piece of information:

<div style="text-align: center">
<img src="imgsL3/listing_screenshot.png" width="800"/>
</div>

<div style="background-color: #C6E2FF; padding: 10px; border-radius: 3px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); width:95%
">
    <strong>▶▶▶ Directions</strong> 
        <ol>
            <li>Use the cell below to extract the individual pieces of information about the apartments. </li>
            <ul>
                <li>Four of the columns are already done for you.</li>
                <li>Extract the data for "furnishing" and "parking".</li>
            </ul>
        <strong>Note</strong>: this part of code will run even if you don't change anything, but the DataFrame will not contain the values.
        </ol>
</div>

In [6]:
# List to store apartment data
apartments = []

# Loop through each grid item and extract the details
for item in grid_items:
    paragraphs = item.find_all("p")
    
    apartment = {
        "number_of_bedrooms": paragraphs[0].text.split(" ")[0].strip(),
        "location": paragraphs[1].text.split(": ")[1].strip(),
        "price": paragraphs[2].text.split(": ")[1].strip(),
        "area": paragraphs[3].text.split(": ")[1].strip(),
        
        ### START CODE HERE ###

        "furnishing": paragraphs[5].text.split(": ")[1].strip(),
        "parking": paragraphs[7].text.split(": ")[1].strip(),
        # Optionally you can extract other columns for practice
        
        ### END CODE HERE ###
    }
    
    apartments.append(apartment)

# Create a pandas DataFrame from the list of apartments
df = pd.DataFrame(apartments)

# Display the DataFrame
df.head()

,number_of_bedrooms,location,price,area,furnishing,parking
0,3,Southern Suburbs,$1272,52 sqm,Unfurnished,Yes
1,3,Central,$6120,154 sqm,Unfurnished,Yes
2,1,Other,$745,34 sqm,Unfurnished,Yes
3,1,Southeastern Suburbs,$1048,43 sqm,Unfurnished,Yes
4,3,Central,$2200,63 sqm,Partially Furnished,Yes


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   number_of_bedrooms  72 non-null     object
 1   location            72 non-null     object
 2   price               72 non-null     object
 3   area                72 non-null     object
 4   furnishing          72 non-null     object
 5   parking             72 non-null     object
dtypes: object(6)
memory usage: 3.5+ KB


As you can see, all of the datatypes are objects. At the very least you would want to cast the `number_of_bedrooms` and `price` to numeric, so that you can use the greater-than operator and you can sort them by price. Aditionally, you would want to check the values in each of the columns to see what they are and how it may affect your search.

In [8]:

# Convert the number_of_bedrooms column to integer
df["number_of_bedrooms"] = df['number_of_bedrooms'].astype("Int64")

In [9]:
# Remove the dollar sign from the price column
df["price"] = df['price'].str.replace("$","")

# Convert the price column to integer
df["price"] = df['price'].astype('int64')

In [ ]:
# Find the unique values in the location column and print them out
print(df['location'].unique())

['Southern Suburbs' 'Central' 'Other' 'Southeastern Suburbs' 'Periphery'
 'Northern Suburbs' 'Western Suburbs']


In [ ]:
# Find the unique values in the parking column and print them out
print(df['parking'].unique())

['Yes' 'No']


In [12]:
### START CODE HERE ###

# Filter the DataFrame for apartments with central location
central_apartments_df = df[df['location'] == 'central']

# Filter the DataFrame for apartments with two or more bedrooms
two_bedroom_apartments_df = central_apartments_df[central_apartments_df["number_of_bedrooms"] >= 2]

# Filter the DataFrame for apartments with parking
apartments_with_parking_df = two_bedroom_apartments_df[two_bedroom_apartments_df['parking'] == "Yes"]

# Sort the DataFrame by price in ascending order
sorted_apartments_df = apartments_with_parking_df.sort_values(by="price")

### END CODE HERE ###

# Get the cheapest five apartments
cheapest_five_apartments_df = sorted_apartments_df.head(5)

# Display the result
cheapest_five_apartments_df

,number_of_bedrooms,location,price,area,furnishing,parking


In [16]:
import pandas as pd
# Grabbing all tables from Wikipedia's population by country page
url = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"
tables = pd.read_html(url)
# Let's see what we've snagged
for index, table in enumerate(tables):
    print(f"Table {index}:\n", table.head(), "\n")

HTTPError: HTTP Error 403: Forbidden

In [17]:
import pandas as pd
import requests

url = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"

# Define a header to look like a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

# Fetch the page content
response = requests.get(url, headers=headers)

# Pass the text content to read_html
tables = pd.read_html(response.text)

for index, table in enumerate(tables):
    print(f"Table {index}:\n", table.head(), "\n")

C:\Users\megha\AppData\Local\Temp\ipykernel_6048\3302976778.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


Table 0:
         Location  Population % of world         Date  \
0          World  8232000000       100%  13 Jun 2025   
1          India  1417492000      17.2%   1 Jul 2025   
2          China  1408280000      17.1%  31 Dec 2024   
3  United States   340110988       4.1%   1 Jul 2024   
4      Indonesia   284438782       3.5%  30 Jun 2025   

  Source (official or from the United Nations) Notes  
0                          UN projection[1][3]   NaN  
1                       Official projection[4]   [b]  
2                         Official estimate[5]   [c]  
3                         Official estimate[6]   [d]  
4                National annual projection[7]   NaN   

Table 1:
   vteLists of countries by population statistics  \
0                                         Global   
1                          Continents/subregions   
2                               Intercontinental   
3                             Cities/urban areas   
4                                Past and future   

In [18]:
import requests
from bs4 import BeautifulSoup

# Digging into BBC's latest headlines
url = "https://www.bbc.com/news"
response = requests.get(url)
soup = BeautifulSoup(response.text)

# Find all article headlines
headlines = soup.find_all('h2')
for headline in headlines:
    print(headline.text.strip())

Venezuela swears in interim president after defiant Maduro pleads not guilty
Cuba defiant as it braces for post-Maduro era
Protests and US warnings shake Iran at its weakest point in years
Canada's Chrystia Freeland to quit as MP and become adviser to Zelensky
Which countries could be in Trump's sights after Venezuela?
Selfies and smiles: South Korea seeks 'new phase' in ties with China
Anonymous cypto gambler made $436,000 on Maduro capture
Skiers create heart-shaped tribute for Switzerland fire victims
US cuts universal childhood vaccine recommendations, including covid and hepatitis
Nvidia unveils 'reasoning' AI technology for self-driving cars
Ten found guilty of cyber-bullying Brigitte Macron
Timothée's shoutout for Kylie Jenner and other moments from Critics' Choice Awards
Car giant Hyundai to use human-like robots in factories
US seizes Maduro
US sharply criticised by foes and friends over Maduro seizure
What is Maduro charged with and what is the evidence?
Misleading videos cla

In [ ]:
Practice Websites for Pandas:

Wikipedia: Jump into any page with tabular data. For example:

Nobel Prize Winners: Discover the laureates across various fields.

Wikipedia -" https://en.wikipedia.org/wiki/List_of_Nobel_laureates"

Olympic Records: Dive into the record holders for the Summer and Winter Games.

Wikipedia - "https://en.wikipedia.org/wiki/List_of_Olympic_records_in_athletics"

UNESCO World Heritage Sites: Learn about sites of cultural and natural importance.

Wikipedia - "https://en.wikipedia.org/wiki/Lists_of_World_Heritage_Sites"

Endangered Species: Review the conservation status of various species.

Wikipedia - "https://en.wikipedia.org/wiki/Lists_of_IUCN_Red_List_endangered_species"

bbc news - "https://www.bbc.com/news"

books to scrape - "https://books.toscrape.com/"